# Deploying ADK agent on Agent Engine from source files

# Deploying ADK agent on Agent Engine from source files

This guide explains how to deploy an AI agent built with the **Google Agent Development Kit (ADK)** to **Agent Engine** (Vertex AI Reasoning Engine) using local source files.

## Summary

Standard deployment for Vertex AI Agent Engine often relies on "on-the-fly" serialization of Python objects, which can fail if the agent contains non-serializable components.

**The Solution:** Use **Source-Based Deployment**. By providing the path to your source code and specifying an entry point, Agent Engine instantiates the object directly in the cloud, bypassing local serialization errors.

---

## Deployment Steps

1. **Environment Initialization:** Set up the Vertex AI SDK with your Project ID and target Location.
2. **Source Preparation:** * Create a local directory (e.g., `sample_agent`) for your logic.
* Define `agent.py`, where `AdkApp` and `LlmAgent` are instantiated.
* Define `requirements.txt` to include `google-adk` and related libraries.


3. **Client Configuration:** Initialize the `vertexai.Client` using `v1beta1` features required for Agent Engine management.
4. **Deployment Configuration:** Define a configuration dictionary mapping the local source to the cloud via `source_packages`, `entrypoint_module`, and `entrypoint_object`.
5. **Validation:** Implement a client to test the deployed agent via asynchronous streaming queries.

---

## Key Considerations

#### 1. Avoiding Serialization Errors

Using `source_packages` and `entrypoint_module` tells Vertex AI to "install and run" your code rather than "copying a memory snapshot". This is the recommended path for agents using complex third-party libraries.

#### 2. Automatic Session Management

When running on Agent Engine, the ADK framework automatically utilizes the `VertexAiSessionService`. You do **not** need to explicitly set or configure this service within your code, as the environment handles state persistence automatically.

#### 3. Dynamic `app_name` via Environment Variables

The `AdkApp` requires the full Reasoning Engine resource name (e.g., `projects/.../reasoningEngines/...`) to manage sessions.

* **Automatic Injection:** Agent Engine automatically sets the `GOOGLE_CLOUD_AGENT_ENGINE_ID` environment variable during deployment.
* **Implementation:** Use `os.environ.get('GOOGLE_CLOUD_AGENT_ENGINE_ID')` to dynamically set the `app_name`.

#### 4. API Versioning

This process utilizes the `v1beta1` API version via `HttpOptions` to access the latest Agent Engine management features.

In [1]:
import vertexai
from google.genai.types import HttpOptions

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [2]:
%%bash
mkdir -p sample_agent
cat <<'EOF' >sample_agent/agent.py
import os
from vertexai.agent_engines import AdkApp
from google.adk.agents import LlmAgent

root_agent = LlmAgent(
    name='frientdly_agent',
    model='gemini-2.5-flash',
    instruction='Be friendly.',
)

# entrypoint_object
app = AdkApp(
    agent=root_agent,
    # app_name should be the full Reasoning Engine resource name.
    app_name=os.environ.get('GOOGLE_CLOUD_AGENT_ENGINE_ID', 'default-app-name'),
)
EOF

cat <<'EOF' >sample_agent/requirements.txt
google-adk==1.21.0
google-cloud-aiplatform==1.132.0
google-genai==1.56.0
EOF

In [3]:
DISPLAY_NAME = 'friendly_agent'

client = vertexai.Client(
    project=PROJECT_ID,
    location=LOCATION,
    http_options=HttpOptions(
        api_version='v1beta1',
        base_url=f'https://{LOCATION}-aiplatform.googleapis.com/'
    ),
)

config = {
        'display_name': DISPLAY_NAME,
        'source_packages': ['sample_agent'],
        'entrypoint_module': 'sample_agent.agent',
        'entrypoint_object': 'app',
        'requirements_file': 'sample_agent/requirements.txt',
        'class_methods': [
            {
                'name': 'async_stream_query',
                'api_mode': 'async_stream',
                'description': 'Stream responses from the agent for a given query.',
                'parameters': {
                    'type': 'object',
                    'properties': {
                        'input': {'type': 'string', 'description': 'The user query'},
                        'config': {'type': 'object', 'description': 'Optional runtime config'}
                    },
                    'required': ['input']
                }
            },
            {
                'name': 'async_create_session',
                'api_mode': 'async',
                'description': 'Create a new managed session.'
            }
        ]
}

remote_agent = client.agent_engines.create(
    config=config,
)

**Wait a few minutes here for the deployed agent to be ready, and continue to the next cell.**

In [4]:
# Chat client to test AdkApp
class ChatClient:
    def __init__(self, app, user_id='default_user'):
        self._app = app
        self._user_id = user_id
        self._session_id = None

    async def async_stream_query(self, message):
        if not self._session_id:
            session = await self._app.async_create_session(
                user_id=self._user_id,
            )
            self._session_id = getattr(session, 'id', None) or session['id']

        result = []
        async for event in self._app.async_stream_query(
            user_id=self._user_id,
            session_id=self._session_id,
            message=message,
        ):
#            print('====')
#            print(event)
#            print('====')
            if ('content' in event and 'parts' in event['content']):
                response = '\n'.join(
                    [p['text'] for p in event['content']['parts'] if 'text' in p]
                )
                if response:
                    print(response)
                    result.append(response)
        return result

In [6]:
client = ChatClient(remote_agent)
_ = await client.async_stream_query('how are you?')

Hello! I'm doing quite well, thank you for asking! 😊

As an AI, I don't have feelings in the way humans do, but I'm fully operational and ready to help you with anything you need.

How are *you* doing today? And is there anything I can assist you with?
